# Reading in Data

In this lesson, we will analyze a data set of COVID-19 cases from Portugal as of May 28, 2020. Let's start by reading in the data.

In [ ]:
import pandas as pd

url = "http://dlsun.github.io/pods/data/covid/"
portugal_data = pd.read_csv(url + "portugal_2020-05-28.csv")
portugal_data

The data is stored in a data type called a `DataFrame`. A `DataFrame` is a two-dimensional object, with rows and columns.

We can access a single column, such as the "sex" column:

In [ ]:
portugal_data["sex"]

We can also access a single row, such as row 10 (which is actually the 11th row because Python uses 0-based indexing):

In [ ]:
portugal_data.loc[10]

A single row or column is stored in a data type called a `Series`. A `Series` is a one-dimensional object.

We can also access multiple columns by passing in a _list_ of column names. The result is necessarily a `DataFrame` because we have multiple rows _and_ multiple columns.

In [ ]:
portugal_data[["sex", "age"]]

# Categorical Data

All three of the columns in the data set are **categorical** (as opposed to quantitative). To summarize a categorical variable, we count the number of observations in each category.

In [ ]:
portugal_data["sex"].value_counts()

A **bar plot** is used to visualize a categorical variable. To make a bar plot, call `.plot.bar()` on the counts that we just calculated. Notice that we can chain commands.

In [ ]:
portugal_data["sex"].value_counts().plot.bar()

**Problem 1.** Summarize and visualize the "fatality" variable. What do you think the values in this column represent?

In [ ]:
# YOUR CODE HERE

**Problem 2.** Summarize and visualize the "age" variable. How do the categories seem to be ordered? Can you figure out a way to get the categories to be ordered naturally?

In [ ]:
# YOUR CODE HERE

## Comparing Distributions

A data set of COVID cases in Italy can be found at the following URL: http://dlsun.github.io/pods/data/covid/italy_2020-05-26.csv.

**Problem 3.** Read in this data into the variable `italy_data` and make a graph of the "age" variable for this data set.

In [ ]:
italy_data = # READ IN DATA

If we want to compare the "age" variable across the two data sets (Portugal and Italy), we will need to plot them on the same graph. We can do this by stacking the counts side by side in a `DataFrame`.

In [ ]:
cases_by_age = pd.concat(
    [portugal_data.age.value_counts(), italy_data.age.value_counts()],
    axis="columns",
    keys=["Portugal", "Italy"]
    ).sort_index()

cases_by_age

Calling `.plot.bar()` on this `DataFrame` will produce a grouped bar plot.

In [ ]:
cases_by_age.plot.bar()

Unfortunately, it is a bit hard to compare the two distributions because there are so many more cases in Italy than there are in Portugal. We can solve this problem by converting the counts into proportions or percentages of the total.

To do this, we need to divide by the total count. This operation is called **normalization**. We can do the normalization when we call `.value_counts()`.

In [ ]:
distributions = pd.concat(
    [portugal_data.age.value_counts(normalize=True), 
     italy_data.age.value_counts(normalize=True)],
     axis="columns",
     keys=["Portugal", "Italy"]
     ).sort_index()

distributions

In [ ]:
distributions.plot.bar()

Or we can do it after-the-fact, by calculating the total of each column and dividing.

In [ ]:
distributions = cases_by_age / cases_by_age.sum()
distributions.plot.bar()

## Relationships between Two Categorical Variables

The most interesting questions usually involve more than one variable.

1. Does fatality differ across sex?
2. Does fatality differ across age?

To summarize more than one categorical variable, we can call `.value_counts()` on a `DataFrame` with multiple columns. 

In [ ]:
fatality_by_sex = portugal_data[["fatality", "sex"]].value_counts()
fatality_by_sex

It is easier to read this if the counts are arranged in a **two-way table**, with one variable indexing rows and the other indexing columns.

In [ ]:
fatality_sex_table = fatality_by_sex.unstack("fatality").sort_index()
fatality_sex_table

We can make a grouped bar plot of these counts, just like we did above.



In [ ]:
fatality_sex_table.plot.bar()

It is difficult to compare fatality rates for men and women because many more women test positive for COVID than men. We can normalize the counts for each sex, in the same way that we normalized the counts for the different countries (Portugal vs. Italy).

First, we need to calculate the total for each sex.


In [ ]:
sex_counts = fatality_sex_table.sum(axis="columns")
sex_counts

Now, we need to divide each row by the corresponding total. This is called the "conditional distribution of fatality given sex".

In [ ]:
fatality_given_sex = fatality_sex_table.divide(sex_counts, axis="rows")
fatality_given_sex.plot.bar()
fatality_given_sex

If there are only two categories in a conditional distribution, then one of the categories is redundant (since the numbers have to add up to 1). For clarity, we can plot just one of these categories.

In [ ]:
fatality_given_sex[1].plot.bar()

We see that men have a higher COVID-19 fatality rate than women.

**Problem 4.** Make a visualization showing the COVID-19 fatality rate for each age group in Portugal.

In [ ]:
# YOUR CODE HERE

## Case Study

We will now compare the Portugal data to Colombia data collected on the same day.

In [ ]:
colombia_data = pd.read_csv(url + "colombia_2020-05-28.csv")
colombia_data

The Colombia data is more raw than the Portugal data. We need to transform some of the variables to make it comparable to the Portugal data.

### Transforming a Categorical Variable

Each row represents a case, but there is no column that corresponds directly to fatality. We will transform the "Estado" (state) column into a fatality column. If a patient died, then their state is marked as "Fallecido" (deceased). 

In [ ]:
colombia_data["Estado"].map({
    "Fallecido": 1
})

There are other states, but because we did not specify their value, they were replaced by NaNs. We will fill these NaNs with 0s.

In [ ]:
colombia_data["fatality"] = colombia_data["Estado"].map({
    "Fallecido": 1
}).fillna(0)

colombia_data

### Transforming a Quantitative Variable Into a Categorical Variable

We also need to convert "Edad" (age), which is a quantitative variable in this data set, into age ranges.

In [ ]:
colombia_data["age"] = pd.cut(
    colombia_data["Edad"], 
    bins=[0, 10, 20, 30, 40, 50, 60, 70, 80, 120],
    labels=["0-9", "10-19", "20-29", "30-39", "40-49", "50-59", "60-69", "70-79", "80+"],
    right=False)

colombia_data

**Problem 5.** Calculate the overall fatality rate for Colombia. Compare it with the rate for Portugal.

In [ ]:
# YOUR CODE HERE

**Problem 6.** Calculate the fatality rates for each age group for Colombia. Make a visualization comparing these rates with the ones for Portugal. Do you notice anything strange? (Compare with your answer to the previous exercise.) Can you explain what is going on?

In [ ]:
# YOUR CODE HERE